In [1]:
import pandas as pd
import numpy as np
import pickle
import re
from gensim.models import FastText, Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

## load data

In [2]:
with open('zebal_version2.pickle', 'rb') as f:
    df = pickle.load(f)

In [3]:
# 가입 인사 제거
df = df[df['board']!='가입 인사 (등업 필수)']
df = df.reset_index()

In [4]:
w2v = Word2Vec.load("word2vec_cbow.model")

c:\users\jinny\miniconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [5]:
ft = FastText.load("FastText_cbow.model")

c:\users\jinny\miniconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [6]:
from gensim.corpora import Dictionary
from gensim.models.tfidfmodel import TfidfModel
from gensim.matutils import sparse2full

In [7]:
docs_dict = Dictionary(df['morphs'])
docs_dict.filter_extremes(no_below=20, no_above=0.2)
docs_dict.compactify()

In [8]:
docs_corpus = [docs_dict.doc2bow(doc) for doc in df['morphs']]
model_tfidf = TfidfModel(docs_corpus, id2word=docs_dict)
docs_tfidf  = model_tfidf[docs_corpus]

In [13]:
docs_vecs   = np.vstack([sparse2full(c, len(docs_dict)) for c in docs_tfidf])

### w2v cbow

In [14]:
tfidf_emb_vecs = np.vstack([w2v.wv[docs_dict[i]] for i in range(len(docs_dict))])

In [15]:
docs_vecs.shape

(28569, 9798)

In [16]:
tfidf_emb_vecs.shape

(9798, 300)

In [17]:
docs_emb = np.dot(docs_vecs, tfidf_emb_vecs)

In [19]:
with open('tfidf_weighted_w2v_cbow_v3.pickle', 'wb') as f:
    pickle.dump(docs_emb, f)

### FastText cbow

In [20]:
tfidf_emb_vecs2 = np.vstack([ft.wv[docs_dict[i]] for i in range(len(docs_dict))])

In [21]:
tfidf_emb_vecs2.shape

(9798, 300)

In [22]:
docs_emb2 = np.dot(docs_vecs, tfidf_emb_vecs2)

In [23]:
docs_emb2.shape

(28569, 300)

In [24]:
with open('tfidf_weighted_fasttext_cbow.pickle', 'wb') as f:
    pickle.dump(docs_emb2, f)

## Custom function to count words in vocab

In [51]:
# 필터링 사전
a=dict(docs_dict.items())
b=[j for (i,j) in a]

In [48]:
a.values()

dict_values(["'", '+', '-', '.(', '..)', '/', '1', '16', '2', '26', '35', '4회', ':', 'MBTI', 'X', 'only', '~', '가능', '가입', '가져오다', '감', '강의', '강퇴', '갖다', '갖추다', '개인', '거주', '계획', '고려', '공격성', '공유', '과도', '관련', '규정', '기타', '깊다', '나가다', '나가야', '나서다', '나이', '낮다', '늑대', '닉네임', '답', '대여', '대화', '돗자리', '드리다', '등', '땐', '또는', '라인', '링크', '만큼', '먹다', '명', '모든', '모임', '무조건', '밉다', '바라다', '바탕', '밤', '방', '번개', '보기', '보내다', '분만', '비', '비공개', '뿐', '사용', '사이버', '사회', '상', '서', '서로', '선호', '성', '성격', '성별', '성장', '성적', '성향', '세', '수다', '수정', '스탭', '시', '시각', '시즌', '시청', '시키다', '신청', '심리', '아래', '안녕하다', '얘기', '언행', '에는', '에니어그램', '엠헬', '연령', '연속', '영상', '영화', '예상', '오프라인', '온도', '온라인', '와', '운영', '의사', '이나', '이슈', '이하', '인간관계', '인맥', '인사', '인원', '일상', '일요일', '자기', '자기개발', '자랑', '자신', '자제', '작성', '장기간', '적절하다', '점', '정치', '제한', '조', '조건', '조절', '조치', '조화', '종교', '종종', '주가', '주로', '주장', '주제', '주최', '중심', '즉흥', '지역', '지키다', '직업', '직장인', '직접', '진', '참가', '참가자', '참여', '처리', '총', '출석', '치우치다', '카드', '카페', '

In [61]:
def imsohot(n):
    result=[]
    for i in n:
        if i in b:
            result.append(i)
    return len(result)

In [60]:
imsohot(["감","강의","강퇴","강태","태건","명진"])

[['감', '강의', '강퇴'], 3]

In [99]:
imsohot2(df['morphs'][0])

['오프라인',
 '톡방',
 "'",
 '강의',
 '온도',
 '시즌',
 '1',
 "'",
 '지역',
 '제한',
 'X',
 '나이',
 '수정',
 '안녕하다',
 '스탭',
 '하얗다',
 '늑대',
 '오프라인',
 '번개',
 '중심',
 '톡방',
 '운영',
 ':',
 '강의',
 '온도',
 '-',
 '시즌',
 '1',
 '비공개',
 '방',
 '-',
 '+',
 '닉네임',
 '+',
 'MBTI',
 '사용',
 '신청',
 '무조건',
 '개인',
 '직접',
 '개인',
 '톡',
 '신청',
 '주제',
 ':',
 'MBTI',
 '에니어그램',
 '바탕',
 '인간관계',
 '성장',
 '통찰',
 '관련',
 '모든',
 '대화',
 '사회',
 '이슈',
 '일상',
 '공유',
 '자기개발',
 '등',
 '얘기',
 '환영',
 '번개',
 '주로',
 '토요일',
 '이나',
 '일요일',
 '예상',
 '/',
 '번개',
 '주최',
 '카페',
 '규정',
 '상',
 '운영',
 '진',
 '가능',
 '.(',
 '한강',
 '또는',
 '돗자리',
 '대여',
 '시키다',
 '먹다',
 '수다',
 '틀어놓다',
 '수다',
 '등',
 '갖다',
 '가져오다',
 '서로',
 '공유',
 '성격',
 '심리',
 '카드',
 '컬러',
 '심리',
 '카드',
 '통해',
 '자신',
 '타인',
 '성향',
 '파고들다',
 '보기',
 '밤',
 '에는',
 '와',
 '서',
 '영화',
 '영상',
 '시청',
 '가능',
 '편안하다',
 '모임',
 '주가',
 '성향',
 '답',
 '호응',
 '즉흥',
 '번개',
 '종종',
 '참가',
 '조건',
 '가입',
 '인사',
 '작성',
 '엠헬',
 '회원',
 '참가자',
 '거주',
 '지역',
 ':',
 '제한',
 'X',
 '번개',
 '지역',
 ':',
 '주로',
 '한강',
 '2',
 '호선',
 '라인'

In [63]:
df['morphs_length'] = df['morphs'].apply(imsohot)

In [64]:
df.head()

,index,url,user,date,board,title,content,reply,view,like,type,total,morphs,morphs_length
0,0,https://cafe.naver.com/ArticleRead.nhn?clubid=...,스탭 하얀늑대 isfp(jish****),2019.05.09. 18:03,[종합] 모임 단톡 공지,오프라인 톡방 개시 '한강의온도 시즌1' 지역제한X 나이수정,안녕하세요 스탭 하얀늑대입니다.오프라인번개를 중심으로한 톡방을운영하고있습니다.▶▶방...,0.0,272,3.0,isfp,오프라인 톡방 개시 '한강의온도 시즌1' 지역제한X 나이수정 안녕하세요 스탭 하얀늑...,"[오프라인, 톡방, 개시, ', 한, 강의, 온도, 시즌, 1, ', 지역, 제한,...",268
1,1,https://cafe.naver.com/ArticleRead.nhn?clubid=...,총무스탭 우노esfp9w8(just****),2019.05.02. 10:34,[종합] 모임 단톡 공지,[오픈 톡방] TEA & MBTI STORY,주제 : 차(TEA) 관련 모임모임 : 차연 및 그 외 TEA 카페 오프라인 모임개...,18.0,413,3.0,esfp,[오픈 톡방] TEA & MBTI STORY 주제 : 차(TEA) 관련 모임모임 :...,"[[, 오픈, 톡방, ], TEA, &, MBTI, STORY, 주제, :, 차, ...",162
2,2,https://cafe.naver.com/ArticleRead.nhn?clubid=...,스탭 하얀늑대 isfp(jish****),2019.03.26. 13:14,공지 사항,우리카페 통계 몇가지 <2월>,우리카페의 통계정보가 가끔 궁금했던지라 보다가 올려봅니다.2월통계▶방문자 성별비율 ...,8.0,936,22.0,isfp,우리카페 통계 몇가지 <2월> 우리카페의 통계정보가 가끔 궁금했던지라 보다가 올려봅...,"[우리, 카페, 통계, 몇, 가지, <, 2월, >, 우리, 카페, 의, 통계, 정...",325
3,3,https://cafe.naver.com/ArticleRead.nhn?clubid=...,총무스탭 앞가마 isfj(apga****),2019.01.03. 22:15,신변 잡기,몇 달동안이나 미루어 왔던 당부의 말씀을 회원님들께 꼭 드리고 싶습니다 (부제: 사...,저는 오래 전부터 본 카페에서 종종 보여왔던 특정 게시글들 및 댓글들을 머리와 가슴...,83.0,"2,057",83.0,isfj,몇 달동안이나 미루어 왔던 당부의 말씀을 회원님들께 꼭 드리고 싶습니다 (부제: 사...,"[몇, 달동, 안이, 나, 미루다, 오다, 당부, 의, 말씀, 을, 회원, 님, 들...",564
4,4,https://cafe.naver.com/ArticleRead.nhn?clubid=...,킁킁estp(mkla****),2019.05.18. 19:23,ISTP ♤ ESTP,Estp 여자 이미지 어떤가요?,참고로 에니어그램은 8w7 입니다~~,0.0,2,0.0,estp,Estp 여자 이미지 어떤가요? 참고로 에니어그램은 8w7 입니다~~,"[Estp, 여자, 이미지, 어떤, 가요, ?, 참고, 로, 에니어그램, 은, 8,...",11


In [ ]:
docs_vecs/df['morphs_legnth']

아니면 tf-idf 벡터에서 0이 아닌 element의 개수를 셀 수도 있을 것

In [93]:
k=pd.DataFrame(docs_vecs)
sum(k.iloc[0]>0)

187